In [1]:
import pandas as pd

## Data: How is the .csv organized?

For each .csv file the following apply:
* each row represents the data of a single vehicle
* the first 10 columns in the 1st row include the columns’ names
* the first 4 columns include information about the trajectory like the unique trackID, the type of vehicle, the distance traveled in meters and the average speed of the vehicle in km/h
* the last 6 columns are then repeated every 6 columns based on the time frequency. For example, column_5 contains the latitude of the vehicle at time column_10, and column­­­_11 contains the latitude of the vehicle at time column_16.
* Speed is in km/h, Longitudinal and Lateral Acceleration in m/sec2 and time in seconds.

In [2]:
# 1. lets blindly try to read the data into a dataframe

df = pd.read_csv("../data/data.csvcsv", sep=";")

FileNotFoundError: [Errno 2] No such file or directory: '../data/data.csvcsv'

The error is caused by the fact that rows do **not** have equal number of columns. 

In [3]:
with open("../data/data.csv", 'r') as file:
    lines = file.readlines()

In [10]:
print(f"The number of rows/lines is {len(lines)}")

The number of rows/lines is 923


In [5]:
print(lines[0]) # column names
print(lines[0].strip('\n').strip().strip(';').split(';')) # columns names as a list

track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time

['track_id', ' type', ' traveled_d', ' avg_speed', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [11]:
lines_as_lists = [line.strip('\n').strip().strip(';').split(';') for line in lines]
len(lines_as_lists)

923

In [7]:
print(f"the number of fields in row 1 is {len(lines_as_lists[1])}, row 2 is {len(lines_as_lists[2])}")

the number of fields in row 1 is 2710, row 2 is 2680


The number of fields in each row is = 4 + 6 * n

* The first 4 unique values are: track_id; type; traveled_d; avg_speed
* The remaining fields are repeated sequences of: 
  * lat_1; lon_1; speed_1; lon_acc_1; lat_acc_1; time_1
  * lat_2; lon_2; speed_2; lon_acc_2; lat_acc_2; time_2
  * ...
  * lat_n; lon_n; speed_n; lon_acc_n; lat_acc_n; time_n

* n depends on the time frequency. It is different for each row.


In [9]:
no_field_max = 0

for row in lines_as_lists:
    if len(row) > no_field_max:
        no_field_max = len(row)

print(f"the maximum number of fields is {no_field_max}")
largest_n = int((no_field_max-4)/6)
print(f"the largest n = {largest_n}")

the maximum number of fields is 122830
the largest n = 20471


We can divide each row into 2 parts:
1.  A list of the first 4 values. 
    - These will correspond to ['track_id', ' type', ' traveled_d', ' avg_speed']
2.  A matrix of size 6*n of the remaining values. 
    - These will correspond to a table with columns [' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time'] and n rows.
    - We will append the 'track_id' value to each row to keep track of the vehicle identity.

In [12]:
cols = lines_as_lists.pop(0)

In [16]:
track_cols = cols[:4]
trajectory_cols = ['track_id'] + cols[4:]

print(track_cols)
print(trajectory_cols)


['track_id', ' type', ' traveled_d', ' avg_speed']
['track_id', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [17]:
track_info = []
trajectory_info = []

for row in lines_as_lists:
    track_id = row[0]

    # add the first 4 values to track_info
    track_info.append(row[:4]) 

    remaining_values = row[4:]
    # reshape the list into a matrix and add track_id
    trajectory_matrix = [ [track_id] + remaining_values[i:i+6] for i in range(0,len(remaining_values),6)]
    # add the matrix rows to trajectory_info
    trajectory_info = trajectory_info + trajectory_matrix


In [18]:
df_track = pd.DataFrame(data= track_info,columns=track_cols)

df_track.head()

,track_id,type,traveled_d,avg_speed
0,1,Car,48.85,9.770344
1,2,Motorcycle,98.09,19.839417
2,3,Motorcycle,63.80,18.228752
3,4,Motorcycle,145.72,26.229014
4,5,Motorcycle,138.01,24.841425


In [20]:
df_trajectory = pd.DataFrame(data= trajectory_info,columns=trajectory_cols)

df_trajectory.head(20)

,track_id,lat,lon,speed,lon_acc,lat_acc,time
0,1,37.977391,23.737688,4.9178,0.0518,-0.0299,0.000000
1,1,37.977391,23.737688,4.9207,-0.0124,-0.0354,0.040000
2,1,37.977391,23.737688,4.9160,-0.0519,-0.0413,0.080000
3,1,37.977390,23.737688,4.9057,-0.0914,-0.0478,0.120000
4,1,37.977390,23.737689,4.8871,-0.1679,-0.0550,0.160000
5,1,37.977389,23.737689,4.8547,-0.2815,-0.0628,0.200000
6,1,37.977389,23.737689,4.8033,-0.4320,-0.0713,0.240000
7,1,37.977388,23.737690,4.7318,-0.5617,-0.0804,0.280000
8,1,37.977388,23.737690,4.6442,-0.6543,-0.0905,0.320000
9,1,37.977388,23.737690,4.5437,-0.7413,-0.1017,0.360000
